In [ ]:
import os
import sys
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing import image
import numpy as np

In [ ]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 
           '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [ ]:
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3

In [ ]:
result_dir = 'results'

In [ ]:
# VGG16
input_tensor = Input(shape=(img_height, img_width, channels))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# FC
fc = Sequential()
fc.add(Flatten(input_shape=vgg16.output_shape[1:]))
fc.add(Dense(256, activation='relu'))
fc.add(Dropout(0.5))
fc.add(Dense(nb_classes, activation='softmax'))

# VGG16とFCを接続
model = Model(input=vgg16.input, output=fc(vgg16.output))

# 学習済みの重みをロード
model.load_weights(os.path.join(result_dir, 'finetuning.h5'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.summary()

In [ ]:
# load test data and make prediction
path = os.path.join('input', 'given', 'test', '*.jpg')
files = sorted(glob.glob(path))

In [ ]:
X_test = []
X_test_id = []
    
for fl in files:
    flbase = os.path.basename(fl)
    img = image.load_img(fl, target_size=(img_rows, img_cols))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    X_test.append(model.predict(x))
    X_test_id.append(flbase)

df = pd.DataFrame.from_records(X_test, index=X_test_id)
df.to_csv('sub.csv')

In [ ]:
from IPython.display import FileLink
FileLink('sub.csv')